# Prepare data for training

- Create a dataset from raw data:
  - (doc, query, relevance)
  - relevance is 0 (not relevant), 1 (relevant), 2 (user-selected)


In [4]:
!pip install polars

  Using cached polars-1.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.2 MB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import pandas as pd
# from datasets import load_dataset
# import matplotlib.pyplot as plt
# import seaborn as sns
# import collections
import random
import numpy as np
import os
import polars as pl

In [2]:
# data = load_dataset("ms_marco", "v1.1", split="train")

README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

c:\Users\kaleb\Code\TwoTowerSearch\env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kaleb\.cache\huggingface\hub\datasets--ms_marco. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [3]:
# def preprocess(text: str) -> list[str]:
#     text = text.lower()
#     text = text.replace(".", " <PERIOD> ")
#     text = text.replace(",", " <COMMA> ")
#     text = text.replace('"', " <QUOTATION_MARK> ")
#     text = text.replace(";", " <SEMICOLON> ")
#     text = text.replace("!", " <EXCLAMATION_MARK> ")
#     text = text.replace("?", " <QUESTION_MARK> ")
#     text = text.replace("(", " <LEFT_PAREN> ")
#     text = text.replace(")", " <RIGHT_PAREN> ")
#     text = text.replace("--", " <HYPHENS> ")
#     text = text.replace("?", " <QUESTION_MARK> ")
#     text = text.replace(":", " <COLON> ")
#     words = text.split()
#     stats = collections.Counter(words)
#     words = [word for word in words if stats[word] > 5]
#     return words

In [16]:
# df_raw = pd.DataFrame(data)

In [17]:
# df_raw.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[]
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[]
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[]
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[]
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[]


In [6]:
# def unwrap_passages(row: pd.Series) -> pd.DataFrame:
#     df_psg = pd.DataFrame(row["passages"])
#     df_psg["query"] = row["query"]
#     df_psg["query_id"] = row["query_id"]
#     df_psg["query_type"] = row["query_type"]
#     # answers = row['answers']
#     # df_psg['answers'] = [answers] * len(df_psg) if answers and any(answers) else [[]] * len(df_psg)
#     return df_psg

In [18]:
# chunks = []
# for i, row in df_raw.iterrows():
#     chunks.append(unwrap_passages(row))
# df = pd.concat(chunks).reset_index(drop=True)

In [36]:
# df.head(1)

,is_selected,passage_text,url,query,query_id,query_type
0,0,"Since 2007, the RBA's outstanding reputation h...",https://en.wikipedia.org/wiki/Reserve_Bank_of_...,what is rba,19699,description


In [38]:
# import os

# # save df to data/processed/flattened_train.parquet
# file_path = os.path.join("data", "processed", "flattened_train.parquet")
# df.to_parquet(file_path)

In [6]:
# import df from data/processed/flattened_train.parquet
file_path = os.path.join("flattened_train.parquet")
df = pd.read_parquet(file_path)

In [7]:
len(df)

676193

In [16]:
dfc = df.copy()

In [17]:
# Rename columns for clarity
dfc = dfc.rename({
    "is_selected": "is_selected",
    "passage_text": "passage_text",
    "url": "url",
    "query": "query",
    "query_id": "query_id",
    "query_type": "query_type"
})

# Convert the Pandas DataFrame to a Polars DataFrame
dfc = pl.from_pandas(pd.DataFrame(dfc))

# Get unique queries
unique_queries = dfc.select("query").unique()["query"].to_list()

# Create empty lists to hold new data
query_list = []
query_id_list = []
relevant_doc_list = []
irrelevant_doc_list = []
relevant_doc_is_selected_list = []

# Iterate through each unique query
num_queries = len(unique_queries)
for idx, query in enumerate(unique_queries):
    # Print progress every 10%
    if idx % (num_queries // 10) == 0:
        print(f"Progress: {idx / num_queries * 100:.1f}% completed")

    # Get all relevant documents for the current query
    query_relevant_docs = dfc.filter(pl.col("query") == query)
    # Get irrelevant documents (documents that belong to other queries)
    irrelevant_docs = dfc.filter(pl.col("query") != query)

    # Sample an irrelevant document for each relevant document
    sampled_irrelevant_docs = irrelevant_docs.sample(len(query_relevant_docs))

    # Combine relevant and irrelevant documents
    query_list.extend([query] * len(query_relevant_docs))
    query_id_list.extend(query_relevant_docs["query_id"].to_list())
    relevant_doc_list.extend(query_relevant_docs["passage_text"].to_list())
    irrelevant_doc_list.extend(sampled_irrelevant_docs["passage_text"].to_list())
    relevant_doc_is_selected_list.extend(query_relevant_docs["is_selected"].to_list())

# Create final Polars DataFrame
final_df = pl.DataFrame({
    "query": query_list,
    "query_id": query_id_list,
    "relevant_doc": relevant_doc_list,
    "irrelevant_doc": irrelevant_doc_list,
    "relevant_doc_is_selected": relevant_doc_is_selected_list
})

Progress: 0.0% completed
Progress: 10.0% completed
Progress: 20.0% completed
Progress: 30.0% completed
Progress: 40.0% completed
Progress: 50.0% completed
Progress: 60.0% completed
Progress: 70.0% completed
Progress: 80.0% completed
Progress: 90.0% completed
Progress: 100.0% completed


In [18]:
final_df.head(10)

query,query_id,relevant_doc,irrelevant_doc,relevant_doc_is_selected
str,i64,str,str,i64
"""what is the two subunits of a …",81114,"""Each ribosome is divided into …","""The appearance of scabs after …",0
"""what is the two subunits of a …",81114,"""The name of the DNA subunit is…","""Bronchospasms are a possible s…",0
"""what is the two subunits of a …",81114,"""Conceptually, the circuitry of…","""Menopause is a natural part of…",0
"""what is the two subunits of a …",81114,"""An arithmetic-logic unit (ALU)…","""gasoline prices in saudi arabi…",0
"""what is the two subunits of a …",81114,"""The two basic units of a CPU, …","""Stefanie Powers is an American…",0
"""what is the two subunits of a …",81114,"""The CPU itself is comprised of…","""1 Kits typically do not includ…",1
"""what is the two subunits of a …",81114,"""The central processing unit is…","""People sold what they purporte…",0
"""what is the two subunits of a …",81114,"""1 The central processing unit(…","""A demonstration of LG's MediaL…",0
"""what is the two subunits of a …",81114,"""Figure 4.1 explains the genera…","""The best-paid 10 percent made …",0


In [20]:
final_df.write_parquet("query_rel_doc.parquet")

In [25]:
# !pip install nltk
# !pip install gensim

In [26]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess

In [27]:
nltk.download("stopwords")
nltk.download("punkt")

# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

def preprocess_list(tokens: list[str]) -> list[str]:
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return tokens


def preprocess_str(text: str) -> list[str]:
    if text is None or pd.isna(text):
        return []

    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = simple_preprocess(
        text, deacc=True
    )  # deacc=True removes accents and punctuations

    tokens = preprocess_list(tokens)
    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [34]:
dfa = final_df.to_pandas()
dfa = dfa[["query_id", "query", "relevant_doc", "irrelevant_doc"]]
dfa.reset_index(inplace=True)

In [35]:
# convert the strings, from each column, into list of tokens
dfa.loc[:, "query_tokens"] = dfa["query"].apply(preprocess_str)
dfa.loc[:, "relevant_document_tokens"] = dfa["relevant_doc"].apply(preprocess_str)
dfa.loc[:, "irrelevant_document_tokens"] = dfa["irrelevant_doc"].apply(
    preprocess_str
)

In [37]:
dfa.to_parquet("list_of_tokens.parquet")

In [2]:
dfa = pd.read_parquet("list_of_tokens.parquet")

In [39]:
!pip install gensim


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import gensim
import torch
import torch.nn as nn

In [4]:
w2v = gensim.models.Word2Vec.load(
    "word2vec-gensim-text8-custom-preprocess.model"
)

vocab = w2v.wv.index_to_key
word_to_idx = {word: i for i, word in enumerate(vocab)}
embeddings_array = np.array([w2v.wv[word] for word in vocab])
embeddings = torch.tensor(embeddings_array, dtype=torch.float32)
print(embeddings.shape)

embedding_layer = nn.Embedding.from_pretrained(embeddings, freeze=True)

torch.Size([74792, 100])


In [5]:
def embed_tokens(tokens: list[str], unknown_tokens: set):
    valid_tokens = [token for token in tokens if token in word_to_idx]
    unknown_tokens.update(set(tokens) - set(valid_tokens))
    if valid_tokens:
        return (
            embedding_layer(
                torch.tensor(
                    [word_to_idx[token] for token in valid_tokens], dtype=torch.long
                )
            ),
            unknown_tokens,
        )
    return torch.tensor([])

In [6]:
unknown_tokens = set()

dfa.loc[:, "query_embedding"] = dfa["query_tokens"].apply(
    embed_tokens, args=(unknown_tokens,)
)
dfa.loc[:, "relevant_document_embedding"] = dfa["relevant_document_tokens"].apply(
    embed_tokens, args=(unknown_tokens,)
)
dfa.loc[:, "irrelevant_document_embedding"] = dfa["irrelevant_document_tokens"].apply(
    embed_tokens, args=(unknown_tokens,)
)

In [7]:
# Remove rows with empty embeddings
dfb = dfa[
    dfa["query_embedding"].apply(
        lambda x: len(x[0]) > 0 if isinstance(x, tuple) else len(x) > 0
    )
    & dfa["relevant_document_embedding"].apply(
        lambda x: len(x[0]) > 0 if isinstance(x, tuple) else len(x) > 0
    )
    & dfa["irrelevant_document_embedding"].apply(
        lambda x: len(x[0]) > 0 if isinstance(x, tuple) else len(x) > 0
    )
]

In [8]:
# print the number of unknown tokens
print(f"Number of unknown tokens: {len(unknown_tokens)}")
# print the first 10 unknown tokens
print(f"Some unknown tokens: {list(unknown_tokens)[:10]}")

Number of unknown tokens: 234255
Some unknown tokens: ['alsothət', 'slumdog', 'modulatedmad', 'γαια', 'crorun', 'tohuman', 'slimacceler', 'cagepotato', 'antifeminist', 'paramu']


In [ ]:
query_embeddings = []
rel_doc_embeddings = []
irrel_doc_embeddings = []

max_query_len = 0
max_rel_doc_len = 0
max_irrel_doc_len = 0

for _, row in dfb.iterrows():
    query_emb = row['query_embedding'][0]
    rel_doc_emb = row['relevant_document_embedding'][0]
    irrel_doc_emb = row['irrelevant_document_embedding'][0]
    
    max_query_len = max(max_query_len, query_emb.shape[0])
    max_rel_doc_len = max(max_rel_doc_len, rel_doc_emb.shape[0])
    max_irrel_doc_len = max(max_irrel_doc_len, irrel_doc_emb.shape[0])
    
    query_embeddings.append(query_emb)
    rel_doc_embeddings.append(rel_doc_emb)
    irrel_doc_embeddings.append(irrel_doc_emb)

# Pad tensors to the maximum length
def pad_tensor(tensor, max_len):
    padding = torch.zeros(max_len - tensor.shape[0], tensor.shape[1])
    return torch.cat([tensor, padding], dim=0)

query_embeddings = torch.stack([pad_tensor(emb, max_query_len) for emb in query_embeddings])
rel_doc_embeddings = torch.stack([pad_tensor(emb, max_rel_doc_len) for emb in rel_doc_embeddings])
irrel_doc_embeddings = torch.stack([pad_tensor(emb, max_irrel_doc_len) for emb in irrel_doc_embeddings])

# Print shapes to verify
print(f"Query embeddings shape: {query_embeddings.shape}")
print(f"Relevant document embeddings shape: {rel_doc_embeddings.shape}")
print(f"Irrelevant document embeddings shape: {irrel_doc_embeddings.shape}")


In [ ]:
# Save the tensors to a file
file_path = os.path.join("PreRNN_embeddings.pt")
torch.save({
    'query_embeddings': query_embeddings,
    'rel_doc_embeddings': rel_doc_embeddings,
    'irrel_doc_embeddings': irrel_doc_embeddings
}, file_path)

print(f"Embeddings saved to {file_path}")

In [ ]:
# To load the data:
# loaded_data = torch.load(file_path)
# query_embeddings = loaded_data['query_embeddings']
# rel_doc_embeddings = loaded_data['rel_doc_embeddings']
# irrel_doc_embeddings = loaded_data['irrel_doc_embeddings']